![Py4Eng](img/logo.png)

# Graphical user interface
## Yoav Ram

# *Tk* GUI

We will be using [*Tk*](http://www.tkdocs.com/), a user interface toolkit that makes it easy to build desktop graphical user interfaces. Tk is cross-platform and it plays nicely with *Matplotlib* and with the Jupyter notebook.

We don't need to install anything, because Tk is packaged with Python, and is accessible via the `tkinter` module which is part of the standard library.

## Hello World!

Let's start with a simple *Hello World!* application - just a window with a button; when the button is clicked, an *Hello World!* dialog comes up. This example follows [Learning IPython for Interactive Computing and Data Visualization](http://ipython-books.github.io/minibook/) by Cyrille Rossant, pg. 85, but modifed to use Tk.

We use the notebook's magic command `%gui` to let the notebook know that we are using Tk, then import the `tkinter` module - note that in Python 2 this module was called `Tkinter`. We also import `tkinter.messagebox`, which has helper functions to easily create... message boxes.

In [1]:
%gui tk
import tkinter
import tkinter.messagebox

Next, we define out main application window, a class we call `HelloWorld`. We add a push-button, with the label `Click me`, and connect it to the method `clicked`. 

We then create a simple layout and show the window (since it's the main window of the application). 

The `clicked` method creates a dialog with an `OK` button which says `Hello World!`.

Finally, we create the Tk application and the main window. Usually we would need to call `app.mainloop()` to start the application mainloop, but Jupyter takes care of that because we called `%gui tk`.

In [2]:
class HelloWorld:
    def __init__(self, master=None):
        self.frame = tkinter.Frame(master=master)
        self.frame.pack()
        
        self.button = tkinter.Button(self.frame)
        # the following could have been given in the init above if your prefer
        self.button["text"] = "Click me"
        self.button["command"] = self.clicked
        self.button.pack()

    def clicked(self):
        tkinter.messagebox.showinfo("Hello world", "You clicked a button.")        

app = tkinter.Tk()
window = HelloWorld(app)

Interestingly, starting the GUI **doesn't block** the notebook (you can see the empty rather than filled circle at the top right of the notebook) which means we can interact with our window through the notebook. This is very useful for testing and debugging.

For example, we can trigger the `clicked` method without actually clicking the button:

In [5]:
window.clicked()

Change the window title and size:

In [6]:
app.geometry('500x50')
app.title("New title")

''

And close the window:

In [7]:
app.destroy()

## PyGubu Designer

When creating more sophisticated application, it's more convinient to work with a designer - a WYSIWYG GUI editor. 

[Pygubu](https://github.com/alejandroautalan/pygubu) is such a tool, allowing us to create and edit *XML* files that have an `.ui` extension and define the layout and design of a *Tk* GUI application and then load these XML files from our Python model code to use the design.

Let's do a simple example before diving into a more sophisticated example. Install *Pygubu* (`conda run -n Py4Eng XXX` runs `XXX` after activating `Py4Eng` as the Python environment):

In [ ]:
!pip install pygubu

Now open the *Pygubu* application on your desktop (the notebook is locked until you close *Pygubu*; if you want to use it in parallel then you should open it from the terminal):

In [19]:
!pygubu-designer


CLINK D:\workspace\Py4Eng\sessions {git}{hg} 
{lamb} D:\Anaconda3\python.exe -m pygubudesigner 
python version: 3.4.4 on win32
pygubu version: 0.9.7.8


*Pygubu* will open and we can start adding widgets to it and design our GUI:

![QtDesigner](img/Pygubu.png)

We'll build a simple app with just a big textbox to write text to and save\load to\from a file.

The design is implemented in `../scripts/notepad.ui` - you can open it in the *Pygubu* (File -> Open...).

![QtDesigner Notepad UI](img/PygubuNotepad.png)

To start a new design you add a `Frame` or `TopLevel` and then you add the widgets you want to it - all from the widget list on the left. Afterwards, you use the widget editor on the bottom to define the widgets' variable name (`id`) and properties such as text, font, and width. 

![QtDesigner Notepad UI](img/PygubuNotepadLayout.png)

Next, you switch to the `Layout` tab to define the layout of the widgets within the frame. The default layout uses a grid system in which you can set the row and column of every widget, their alignment within their grid cell (`sticky`), horizontal and vertical padding (`padx` and `pady`). You can also have one widget span multiple rows or columns using `rowspan` and `colspan` (like I did with the textbox).

We can save our design (File -> Save...) it's saved as an XML file:

In [8]:
%less ../scripts/notepad.ui

Once we have a nice design `.ui` file, we can use it in our application code

In [9]:
import pygubu

class Notepad:
    def __init__(self, master=None): 
        # Create a pygubu builder
        self.builder = builder = pygubu.Builder()
        # Load design file
        builder.add_from_file('../scripts/notepad.ui')
        # Create the mainwindow widget using a master as parent
        self.mainwindow = builder.get_object('mainwindow', master)


app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

Of course, to make this interactive we need to implement button callbacks and other UI logic.
We can define the button's callbacks in *Pygubu* (the `command` property), but I find it's better to separate design and functionality, so we will configure the callbacks from the code:

In [10]:
import pygubu

class Notepad:
    def __init__(self, master=None): 
        # Create a pygubu builder
        self.builder = builder = pygubu.Builder()
        # Load design file
        builder.add_from_file('../scripts/notepad.ui')
        # Create the mainwindow widget using a master as parent
        self.mainwindow = builder.get_object('mainwindow', master)
        # Get a button and set it's callback
        self.saveButton = builder.get_object('saveButton', master)
        self.saveButton["command"] = self.clicked

        
    def clicked(self):
        tkinter.messagebox.showinfo("Click", "You clicked a button.")
                

app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

## Save button

We start with the save button. We need to:

- implement a new method, `save` that 
  - reads the text from the editor
  - reads the filename from the filename textbox
  - opens a file and write the text
  - catches exceptions and reports them with a dialog (like the first example in this session)
- connect the method to the button

In [13]:
class Notepad:
    def __init__(self, master=None): 
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.saveButton["command"] = self.save

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
            
app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

Note that while the window is open, you can introspect it in the notebook:

In [14]:
print(window.filenameEdit.get())

tmp.txt


In [15]:
fname = window.filenameEdit.get()
%less $fname

## Load button

Next, we write a `load` method, which
- reads a filename from the filename textbox
- reads the text from the file
- puts the text in the editor
We then connect method to the load button.

In [16]:
class Notepad:
    def __init__(self, master=None): 
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.loadButton = builder.get_object('loadButton', master)
        self.saveButton["command"] = self.save
        self.loadButton["command"] = self.load

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
    
    def load(self):
        filename = self.filenameEdit.get()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            self.textEdit.delete(1.0, tkinter.END)
            self.textEdit.insert(1.0, text)
        
app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

In [18]:
%pwd

'/Users/yoavram/Work/Py4Eng/sessions'

In [19]:
print(window.textEdit.get(1.0, tkinter.END))

Hello Python!!!





## Browser button

Lastly, we will implement the browse button that will open a file dialog and save the result to the filename textbox. Let's first experiment:

In [20]:
import tkinter.filedialog
filename = tkinter.filedialog.askopenfilename(
    defaultextension='.txt', 
    filetypes=[('all files', '.*'), ('text files', '.txt')],
#     parent
    title="Choose file"
)
print(filename)

/Users/yoavram/Work/Py4Eng/data/A Tale of Two Cities.txt


In [21]:
import tkinter.filedialog
import pygubu

class Notepad:
    def __init__(self, master=None): 
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.loadButton = builder.get_object('loadButton', master)
        self.browseButton = builder.get_object('browseButton', master)
        self.saveButton["command"] = self.save
        self.loadButton["command"] = self.load
        self.browseButton["command"] = self.browse

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
    
    def load(self):

        filename = self.filenameEdit.get()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            self.textEdit.delete(1.0, tkinter.END)
            self.textEdit.insert(1.0, text)

        
    def browse(self):
        filename = tkinter.filedialog.askopenfilename(
            defaultextension='.txt', 
            filetypes=[('all files', '.*'), ('text files', '.txt')],
            parent=self.mainwindow,
            title="Choose file"
        )
        self.filenameEdit.delete(0, tkinter.END)
        self.filenameEdit.insert(0, filename)
                
app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

That's it, we have our notepad application.

## Exercise

Add a button called "All Caps" that changes the text to uppercase (using `str.upper()`).

In [23]:
import tkinter.filedialog
import pygubu

class Notepad:
    def __init__(self, master=None): 
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.loadButton = builder.get_object('loadButton', master)
        self.browseButton = builder.get_object('browseButton', master)
        self.allCapsButton = builder.get_object('allCapsButton', master)
        self.saveButton["command"] = self.save
        self.loadButton["command"] = self.load
        self.browseButton["command"] = self.browse
        self.allCapsButton["command"] = self.all_caps

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
    
    def load(self):
        filename = self.filenameEdit.get()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            self.textEdit.delete(1.0, tkinter.END)
            self.textEdit.insert(1.0, text)
        
    
    def all_caps(self):
        text = self.textEdit.get(1.0, tkinter.END)
        text = text.upper()
        self.textEdit.delete(1.0, tkinter.END)
        self.textEdit.insert(1.0, text)

        
    def browse(self):
        filename = tkinter.filedialog.askopenfilename(
            defaultextension='.txt', 
            filetypes=[('all files', '.*'), ('text files', '.txt')],
            parent=self.mainwindow,
            title="Choose file"
        )
        self.filenameEdit.delete(0, tkinter.END)
        self.filenameEdit.insert(0, filename)
                
app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

# Scheduling and binding

Let's add a "Word Count" label that displays the number of words in the notepad. We easily write a method `update_word_count`

In [24]:
class Notepad:
    def __init__(self, master=None): 
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.counterLabel = builder.get_object('counterLabel', master)

    def update_word_count(self):
        text = self.textEdit.get(1.0, tkinter.END)
        words = text.split()
        word_count = len(words)
        self.counterLabel['text'] = '{:d}'.format(word_count)
                
app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

We can update the word count label by calling `update_word_count`:

In [26]:
window.update_word_count()

However, we would like to do it automatically every second. The first instinct might be to use a thread, but threads cannot change widget; widgets must be changed from the mainloop (which Jupyter notebook starts for use, but with a regular application we would call `app.mainloop()` to start it). 

However, Tk has a good alternative - we can give a timed callback to the mainloop using `app.after`.

In [27]:
class Notepad:
    def __init__(self, master=None):
        self.master = master
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.counterLabel = builder.get_object('counterLabel', master)

        self.update_word_count()
        
    ### methods removed for brevity, see sessions/gui.ipynb or scripts/notepad.py
    def update_word_count(self):
        text = self.textEdit.get(1.0, tkinter.END)
        words = text.split()
        word_count = len(words)
        self.counterLabel['text'] = '{:d}'.format(word_count)
        self.master.after(1000, self.update_word_count)

app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

An alternative is to update the counter on every key stroke using by [binding](http://effbot.org/tkinterbook/tkinter-events-and-bindings.htm) `update_word_count` to key events on `textEdit`:

In [28]:
class Notepad:
    def __init__(self, master=None):
        self.master = master
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.counterLabel = builder.get_object('counterLabel', master)

        self.textEdit.bind('<Key>', self.update_word_count)
        
    ### methods removed for brevity, see sessions/gui.ipynb or scripts/notepad.py
    def update_word_count(self, *args):
        text = self.textEdit.get(1.0, tkinter.END)
        words = text.split()
        word_count = len(words)
        window.counterLabel['text'] = '{:d}'.format(word_count)

app = tkinter.Tk()
app.title("Notepad")
window = Notepad(app)

# Tk and Matplotlib

[Matplotlib has support for Tk](http://matplotlib.org/examples/user_interfaces/embedding_in_tk.html), so that we can put a plot inside a Tk widget `QWidget`,

We have some importing to do. We first import `matplotlib` and set it to use the `TkAgg`. It's important to do this before any other matplotlib-related import.
We then import from matplotlib a Tk-specialized canvas and navigation toolbar (the latter is optional, used for zomming and tilting).

We also import NumPy and Seaborn. If the GUI crashes when using `FigureCanvas`, try to uninstall Matplotlib and install it again, possibly by downloading a wheel from [Gohlke's website](http://www.lfd.uci.edu/~gohlke/pythonlibs/#matplotlib) (in some Matplotlib builds the Tk extension - `_tkagg.pyd` - is missing).

In [4]:
%gui tk
import tkinter
import matplotlib
print("Matplotlib version:", matplotlib.__version__)
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg as FigureCanvas
from matplotlib.backends.backend_tkagg import NavigationToolbar2TkAgg as NavigationToolbar

import numpy as np
import seaborn as sns
sns.set(
    style='white',
    palette='muted'
)

Matplotlib version: 1.5.2


In [7]:
class MainWindow:
    def __init__(self, master=None):
        
        self.fig = matplotlib.figure.Figure(figsize=(6,4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvas(self.fig, master=master)
        self.canvas.get_tk_widget().pack()

        self.mpl_toolbar = NavigationToolbar(self.canvas, master)
        self.canvas._tkcanvas.pack()

app = tkinter.Tk()
app.wm_title("Tk & Matplotlib")
window = MainWindow(app)

In [8]:
window.ax.plot(range(10))
window.canvas.draw()

Now we can plot to the window from the notebook. Most of this code is the same as we always do with matplottlib, only that:
- we already have `fig` and `ax` objects, defined in the `__init__` above cell
- when we are done plotting we need to call `window.canvas.draw()` to make it update the GUI.

Here's a more complex plot:

In [4]:
x = np.linspace(0, 2 * np.pi, 100)
y1 = np.sin(x)
y2 = np.cos(x)

window.ax.plot(x, y1)
window.ax.plot(x, y2)
window.ax.set(
    xlabel='x',
    ylabel='y',
    xlim=(x.min(), x.max()),
    ylim=(y1.min(), y1.max()),
)
window.fig.tight_layout()
window.canvas.draw()

## Figure events

We'd like to be able to act when the user initiates [events on the figure](http://matplotlib.org/users/event_handling.html). 

We'll start with a simple event: the user clicks on the plot and the app prints the event details to the console:

In [5]:
def on_click(event):
    print(event)

This `onclick` expects a [`MouseEvent`](http://matplotlib.org/api/backend_bases_api.html#matplotlib.backend_bases.MouseEvent) instance that has several attributes:

- `button` is the name of the button (1 for left click, 2 for right click)
- `x`, `y` are the pixels coordinates
- `xdata`, `ydata` are the data coordinates
- `dblclick` if the click was a double click
- `inaxes` the axes in which the click occured; use to change the plot
- `canvas` the canvas in which the click occured; use to update the plot with `canvas.draw()`

We connect this callback/event handler to the canvas object of the figure:

In [6]:
cid = window.fig.canvas.mpl_connect('button_press_event', on_click)

MPL MouseEvent: xy=(340,252) xydata=(3.27719352266,0.212431383747) button=1 dblclick=False inaxes=Axes(0.126944,0.156667;0.843056x0.780783)
MPL MouseEvent: xy=(293,170) xydata=(2.69338520087,-0.312616425777) button=1 dblclick=False inaxes=Axes(0.126944,0.156667;0.843056x0.780783)
MPL MouseEvent: xy=(480,193) xydata=(5.01619703436,-0.165346918227) button=1 dblclick=False inaxes=Axes(0.126944,0.156667;0.843056x0.780783)
MPL MouseEvent: xy=(172,153) xydata=(1.19038930861,-0.421467800922) button=1 dblclick=False inaxes=Axes(0.126944,0.156667;0.843056x0.780783)
MPL MouseEvent: xy=(172,153) xydata=(1.19038930861,-0.421467800922) button=1 dblclick=False inaxes=Axes(0.126944,0.156667;0.843056x0.780783)
MPL MouseEvent: xy=(172,153) xydata=(1.19038930861,-0.421467800922) button=1 dblclick=True inaxes=Axes(0.126944,0.156667;0.843056x0.780783)


To disconnect the callback, you can call (you should do this before supplying a different callback):

In [7]:
window.fig.canvas.mpl_disconnect(cid)

## Example

Let's do something more interesting: when the user double clicks the plot, we will add a black circle marker at that point:

In [8]:
def on_click_marker(event):
    if event.dblclick:
        x, y = event.xdata, event.ydata
        ax = event.inaxes
        ax.plot(x, y, 'ko')
        event.canvas.draw()

In [9]:
cid = window.fig.canvas.mpl_connect('button_press_event', on_click_marker)

In [17]:
window.fig.canvas.mpl_disconnect(cid)

## Exercise

We also want that when the user clicks the right button mouse on the plot, a straight line will be plotted from the origin to where the mouse is at.

# Polygon

In [10]:
class Polygonization:
    def __init__(self, master=None):
        self.master = master
        self.frame = tkinter.Frame(master)
        self.frame.pack(padx=15,pady=15)
        # mpl stuff
        self.fig = matplotlib.figure.Figure(figsize=(6, 4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvas(self.fig, master=self.frame)
        self.canvas.get_tk_widget().pack()
        self.mpl_toolbar = NavigationToolbar(self.canvas, self.master)
        self.canvas._tkcanvas.pack()
        # callbacks
        self.btn_prs_cid = self.fig.canvas.mpl_connect('button_press_event', self.mouse_handler)
        self.btn_rls_cid = self.fig.canvas.mpl_connect('button_release_event', self.mouse_handler)
        self.btn_mtn_cid = self.fig.canvas.mpl_connect('motion_notify_event', self.mouse_handler)
        # draw plot
        x = np.linspace(0, 2 * np.pi, 100)
        y = np.sin(x)
        self.ax.plot(x, y)

    
    def mouse_handler(self, event):
        if not event.dblclick and event.button == 1 and event.inaxes:
            ax = event.inaxes
            if event.name == 'button_press_event':
                self.path = ax.plot([], [], 'k-')[0]
            xdata, ydata = self.path.get_data()
            x, y = event.xdata, event.ydata            
            xdata = np.append(xdata, x)
            ydata = np.append(ydata, y)
            if event.name == 'button_release_event':
                xdata = np.append(xdata, xdata[0])
                ydata = np.append(ydata, ydata[0])
            self.path.set_data(xdata, ydata)
            if event.name == 'button_release_event':                
                data = np.array(self.path.get_data())
                poly = plt.Polygon(data.T, alpha=0.3)                
                ax.add_artist(poly)
            self.canvas.draw()

    
app = tkinter.Tk()
app.wm_title("Polygonization")
window = Polygonization(app)

In [12]:
window.path.get_data()

(array([ 3.91397849,  3.91397849,  3.91397849,  4.00430108,  4.01935484,
         4.10967742,  4.26021505,  4.71182796,  5.13333333,  5.28387097,
         5.56989247,  5.7655914 ,  6.0516129 ,  6.17204301,  6.15698925,
         5.4344086 ,  5.01290323,  4.59139785,  4.15483871,  3.67311828,
         3.67311828,  3.91397849]),
 array([ 0.     ,  0.0125 ,  0.14375,  0.33125,  0.36875,  0.41875,
         0.48125,  0.59375,  0.64375,  0.64375,  0.61875,  0.56875,
         0.44375,  0.375  ,  0.1    , -0.4375 , -0.675  , -0.78125,
        -0.75   , -0.39375, -0.39375,  0.     ]))

# References

-  [TkDocs](http://www.tkdocs.com/)
- [Tkinter 8.5 reference](http://infohost.nmt.edu/tcc/help/pubs/tkinter/web/index.html)
- [Learning IPython for Interactive Computing and Data Visualization](http://ipython-books.github.io/minibook/) by Cyrille Rossant, pg. 85 (Qt example, but a good tutorial).
- [Pygubu](https://github.com/alejandroautalan/pygubu) - Tkinter designer
- [Embedding Matplotlib plots in Tk applications](http://matplotlib.org/examples/user_interfaces/embedding_in_tk.html)
- [Matplotlib: event handling and picking](http://matplotlib.org/users/event_handling.html)
- Jake Vanderplas's [Minesweeper in Matplotlib](https://jakevdp.github.io/blog/2012/12/06/minesweeper-in-matplotlib/), [Quaternions and Key Bindings: Simple 3D Visualization in Matplotlib](http://jakevdp.github.io/blog/2012/11/24/simple-3d-visualization-in-matplotlib/) and [3D Interactive Rubik's Cube in Python](http://jakevdp.github.io/blog/2012/11/26/3d-interactive-rubiks-cube-in-python/) are amazing examples of what can be done with Matplotlib beyond simple plots and using a GUI.

## Colophon
This notebook was written by [Yoav Ram](http://www.yoavram.com) and is part of the _Python for Engineers_ course.

The notebook was written using [Python](http://pytho.org/) 3.4.4, [IPython](http://ipython.org/) 4.0.3 and [Jupyter](http://jupyter.org) 4.0.6.

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)